In [ ]:
import os
from langgraph.graph import StateGraph,START,END,MessagesState
from langgraph.prebuilt import ToolNode
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage,ToolMessage,AIMessage      
from langchain_openai import AzureChatOpenAI,AzureOpenAIEmbeddings
from dotenv import load_dotenv,find_dotenv
from IPython.display import Image

load_dotenv(find_dotenv(),override=True)
endpoint = os.getenv("ENDPOINT_URL")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version="2025-01-01-preview"

llm = AzureChatOpenAI(azure_endpoint=endpoint,api_key=subscription_key,api_version=api_version)

@tool
def currentTemperature(country:str) -> int :
    '''current temperature of the country'''
    if country.startswith("i"):
         return 10
    elif country.startswith("u"):
        return 20
    else:
        return 30

tool_mapping={
    "currentTemperature":currentTemperature
}
tools=[currentTemperature]

def call_model(state:MessagesState):
    messages=state["messages"]
    result=llm.invoke(messages)
    return {"messages": [result]}

def should_continue(state:MessagesState):
    messages=state["messages"]
    last_message=messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

graph=StateGraph(MessagesState)
tool_node=ToolNode(tools)

graph.add_node("agent",call_model)
graph.add_node("tools",tool_node)
graph.add_edge(START,"agent")
graph.add_conditional_edges("agent",should_continue)
graph.add_edge("tools","agent")

runnable=graph.compile()
Image(runnable.get_graph().draw_mermaid_png())

